In [ ]:
#General Libraries:
import pandas as pd
import numpy as np
import os

#Web Scrapping Libraries:
from urllib.request import urlopen
from bs4 import BeautifulSoup

#Error Handling 
import sys
import traceback

#Seeting up folder path:
os.chdir('C:/Users/mohdn/OneDrive/Desktop/Nadeem/DataSets/Reviews_Tool/')

US - B0792K2BK6
UK - B0792KWK57
India - B0792KTHKK
Canada - B0792JYXZK
Australia - B0792KRW2J

In [ ]:
country = input('''Choose the country from the list :
                1 - US,
                2 - UK,
                3 - India,
                4 - Canada,
                5 - Australia
                ''')

product = input('Enter the product id :')
page_count = input('''Enter the amount of pages to scrap
                    Minimum - 1
                    Maximum - 500
                    ''')

In [ ]:
#Creating the dataframe to store the scrapped values

dataframe_comp=pd.DataFrame()

#Scraping Code:

e='Exception message : HTTP Error 503: Service Unavailable'
i=1

while (i <= int(page_count) and e==('Exception message : HTTP Error 503: Service Unavailable')):
    try:
        if country == '1' or 'US':
            html = 'https://www.amazon.com/Echo-Dot-3rd-Gen-speaker/product-reviews/{}/ref=cm_cr_arp_d_paging_btm_next_{}?ie=UTF8&reviewerType=all_reviews&pageNumber={}'.format(product,i,i)
            country ='US'
        if country == '2'or 'UK':
            html = 'https://www.amazon.co.uk/Echo-Dot-3rd-Gen-Charcoal/product-reviews/{}/ref=cm_cr_dp_d_show_all_btm_next_{}?ie=UTF8&reviewerType=all_reviews&pageNumber={}'.format(product,i,i)
            country ='UK'
        if country == '3' or 'India':
            html = 'https://www.amazon.in/All-new-Echo-Dot-3rd-Gen/product-reviews/{}/ref=cm_cr_dp_d_show_all_btm_next_{}?ie=UTF8&reviewerType=all_reviews&pageNumber={}'.format(product,i,i)
            country ='India'
        if country == '4' or 'Canada':
            html = 'https://www.amazon.ca/All-new-Echo-Dot-3rd-Gen/product-reviews/{}/ref=cm_cr_dp_d_show_all_btm_next_{}?ie=UTF8&reviewerType=all_reviews&pageNumber={}'.format(product,i,i)
            country ='Canada'
        if country == '5' or ' Australia':
            html = 'https://www.amazon.com.au/All-new-Echo-Smart-speaker-Alexa/product-reviews/{}/ref=cm_cr_dp_d_show_all_btm_next_{}?ie=UTF8&reviewerType=all_reviews&pageNumber={}'.format(product,i,i)
            country =' Australia'
        
        html = urlopen(html)
        soup = BeautifulSoup(html,"lxml")
        html = soup.prettify('utf-8')
        names = []
        for span in soup.findAll('span',attrs={'class': 'a-profile-name'}):
            name = span.text.strip()
            names.append(name)
        names =names [2::1]
        dataframe = pd.DataFrame({'Country':country,'Names':names})
        dataframe_comp = dataframe_comp.append(dataframe,ignore_index=True)
        i = i+1
        
    except BaseException as ex:
        # Get current system exception
        ex_type, ex_value, ex_traceback = sys.exc_info()

        # Extract unformatter stack traces as tuples
        trace_back = traceback.extract_tb(ex_traceback)

        # Format stacktrace
        stack_trace = list()

        for trace in trace_back:
            stack_trace.append("File : %s , Line : %d, Func.Name : %s, Message : %s" % (trace[0], trace[1], trace[2], trace[3]))

            #print("Exception type : %s " % ex_type.__name__)
            #print("Exception message : %s" %ex_value)
            #print("Stack trace : %s" %stack_trace)
        e = ("Exception message : %s" %ex_value)